# Download Crossref Data

This notebook downloads data from the Crossref API and stores it in a SQLite database.

In [2]:
import os

base_dir = "data/orkg/"

In [3]:
import json

# Read name.json data in directory
def read_json(filename: str):
    json_file = os.path.join(filename)
    with open(json_file) as f:
        img_annotations = json.load(f)

    return img_annotations


# Pretty print json data to console
def print_json(tag: str, data: any):
    print(tag, json.dumps(data, indent=2, sort_keys=True))


# Pretty print json data to file
def write_json(filename: str, data: any):
    with open(filename, "w") as file:
        json.dump(data, file, indent=2, sort_keys=True)

In [4]:
import pandas as pd

meta_infos_raw = read_json(os.path.join(base_dir, "meta_infos.json"))
meta_infos = pd.DataFrame.from_dict(meta_infos_raw)
meta_infos

,author,doi,paper_ids,publication month,publication year,publisher,title,url
0,"[J Wu, H Huang]",10.1109/vetecf.2005.1558414,R26672,,2005,,A probabilistic clustering algorithm in wirele...,
1,"[William A Sethares, Chih-Yu Wen]",10.1155/wcn.2005.686,R26676,12,2005,,Automatic Decentralized Clustering for Wireles...,
2,"[A William, Sethares, Chih-Yu Wen]",10.1109/jsen.2013.2249659,R26679,,2013,,Distributed clustering with directional antenn...,
3,"[Xiaozong Yang, Yoohwan Kim, Ling Wang, Yan Jin]",10.1016/j.comnet.2007.10.005,R26682,2,2008,,EEMC: An energy-efficient multi-level clusteri...,
4,"[A Savvides, R Virrankoski]",10.1109/mahss.2005.1542850,R26687,,,,TASC: topology adaptive spatial clustering for...,
...,...,...,...,...,...,...,...,...
26838,David Nolte,10.1026/2191-9186/a000176,R576013,10,2014,Frühe Bildung,Eine Frage der Medienkompetenz?: Bedingungen m...,http://dx.doi.org/10.1026/2191-9186/a000176
26839,"[Rath, M, Marci-Boehncke, G. ]",,R576016,,2013,,Kinder-Medien-Bildung: eine Studie zu Medienko...,
26840,"[Anja Pielsticker, K. Keller, Henrike Friedric...",,R576018,,2012,,Chancen und Potenziale digitaler Medien zur Um...,
26841,"[Niels Brüggen, Valerie Jochim, A. Oberlinner,...",,R576020,,2018,,Digitale Medien in Kindertageseinrichtungen: M...,


In [5]:
import sqlite3

database_path = os.path.join(base_dir, "datalake.db")

# Open a connection to the SQLite database file
db = sqlite3.connect(database_path)

# Create a cursor object to execute SQL statements
cursor = db.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS crossrefs (id INTEGER PRIMARY KEY, doi TEXT, title Text, data JSON)")

# cursor.execute("SELECT * FROM users")
# rows = cursor.fetchall()
#
# for row in rows:
#     id, name = row
#     # Process the retrieved data
#
# # Commit the changes and close the connection
# conn.commit()
# conn.close()

In [6]:
import requests
from fuzzywuzzy import fuzz

def crossref_by_doi(doi: str):
    url = "https://api.crossref.org/works/" + doi
    response = requests.get(url, timeout=5)

    if not response.ok:
        return None

    message = response.json()["message"]
    return message

def crossref_by_title(title: str):
    url = "https://api.crossref.org/works?query.bibliographic=" + title
    response = requests.get(url, timeout=5)

    if not response.ok:
        return None

    message = response.json()["message"]

    for item in message.get('items', []):
        if not item.get("title", ""):
            continue

        if title.lower() == item.get('title', '')[0].lower():
            return item
        elif fuzz.ratio(title.lower(), item.get('title', '')[0].lower()) > 95:
            return item

    return None

In [7]:
def crossref_exists(doi: str, title: str):
    cursor.execute("SELECT * FROM crossrefs WHERE doi = ? OR title = ?",
                   (doi, title))
    rows = cursor.fetchall()
    return len(rows) > 0

def store_crossref_data(doi: str, title: str, data: dict):
    cursor.execute("INSERT INTO crossrefs (doi, title, data) VALUES (?, ?, ?)", (doi, title, json.dumps(data)))
    db.commit()

In [10]:
from tqdm import tqdm

for index, row in tqdm(meta_infos.iterrows(), total=len(meta_infos)):
    doi = row["doi"]
    title = row["title"]

    # If doi is not a string, skip
    if not isinstance(doi, str):
        print(f"Skipping {index}: {title} ({doi})")
        continue

    if crossref_exists(doi, title):
        continue

    # print(f"Processing {index}: {title} ({doi})")

    if doi is not None:
        data = crossref_by_doi(doi)
    else:
        data = crossref_by_title(title)

    if data is not None:
        store_crossref_data(doi, title, data)
    else:
        print(f"Could not find {index}: {title} ({doi})")

    # break

  0%|          | 119/26843 [00:01<07:59, 55.73it/s]

Could not find 109: A clustering method for energy efficient routing in wireless sensor networks (10.5555/1353572.1353596)


  2%|▏         | 596/26843 [00:06<07:08, 61.28it/s] 

Could not find 579: Wie weit fliegt ein Skispringer, der mit 72 km/h abspringt? (10.5446/40456)
Could not find 601: Implementing LOINC – Current Status and Ongoing Work at a Medical University (10.3233/SHTI190806)


  2%|▏         | 606/26843 [00:07<15:47, 27.70it/s]

...

Could not find 3643: A stochastic model for designing last mile relief networks (10.1287/trsc,2015.0621)


 14%|█▎        | 3674/26843 [01:06<08:36, 44.87it/s]

Could not find 3656: Stochastic network models for logistics planning in disaster relief (10.1061/j.ejor.2016.04.041)


 15%|█▍        | 3960/26843 [01:08<06:37, 57.54it/s] 


KeyboardInterrupt: 

In [11]:
db.close()